# Grid2x2 using RLlib

In [1]:
import csv
import os

import ray
from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility
from ray.tune.registry import register_env

from envs import MultiAgentSumoEnv
from observation import Grid2x2ObservationFunction
from reward_functions import combined_reward

In [2]:
import random
import numpy as np
import torch

TEST_NUM = 6
SEED = 23423  # default SUMO seed no.
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

ENV_NAME = "grid2x2"
assert not os.path.exists(os.path.join("ray_checkpoints",ENV_NAME,f"test_{TEST_NUM}"))

In [3]:
def train_env_creator(args):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid2x2ObservationFunction,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(**env_params)
    return env

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig

# From https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/ppo/atari-ppo.yaml

train_env = MultiAgentEnvCompatibility(train_env_creator({}))

config: PPOConfig
config = (
    PPOConfig()
    .environment(env=ENV_NAME)
    .framework(framework="torch")
    .rollouts(
        rollout_fragment_length=100,
        num_rollout_workers=10,
    )
    .training(
        lambda_=0.95,
        kl_coeff=0.5,
        clip_param=0.1,
        vf_clip_param=10.0,
        entropy_coeff=0.01,
        train_batch_size=1000,
        sgd_minibatch_size=100,
        num_sgd_iter=10,
    )
    .evaluation(
        evaluation_duration=1,
        evaluation_num_workers=1,
        evaluation_sample_timeout_s=300,
    )
    .debugging(seed=SEED)
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "1")))
    .multi_agent(
        policies=set(train_env.env.ts_ids),
        policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
    )
    .fault_tolerance(recreate_failed_workers=True)
)

delta_wait_time_reward 0.7


In [5]:
csv_dir = os.path.join("outputs",ENV_NAME,f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

## Play Untrained Agent

In [14]:
def eval_env_creator(csv_path, tb_log_dir):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid2x2ObservationFunction,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(eval=True, csv_path=csv_path, tb_log_dir=tb_log_dir, **env_params)
    return env

In [8]:
ray.init()

csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_untrained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure",])

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

2023-06-04 10:38:57,155	INFO worker.py:1625 -- Started a local Ray instance.


In [9]:
algo = config.build()

2023-06-04 10:39:00,708	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-04 10:39:00,742	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=31788) delta_wait_time_reward 0.7


(RolloutWorker pid=31788) 2023-06-04 10:39:07,593	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.


(RolloutWorker pid=31788) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                      


2023-06-04 10:39:11,484	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=15804) delta_wait_time_reward 0.7 [repeated 10x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=30340) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 9x across cluster]


(RolloutWorker pid=15804) 2023-06-04 10:39:15,486	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-04 10:39:15,579	INFO trainable.py:172 -- Trainable.setup took 14.840 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [10]:
algo.evaluate()

Step #3600.00 (1ms ~= 1000.00*RT, ~223000.00UPS, TraCI: 265ms, vehicles TOT 1282 ACT 223 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~223000.00UPS, TraCI: 265ms, vehicles TOT 1282 ACT 223 BOT 3 ACT 3 BUF 0)                      


{'evaluation': {'episode_reward_max': -411220.48761799914,
  'episode_reward_min': -411220.48761799914,
  'episode_reward_mean': -411220.48761799914,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'1': -102445.36977694454,
   '2': -111154.78071310197,
   '5': -96249.65747978249,
   '6': -101370.67964817032},
  'policy_reward_max': {'1': -102445.36977694454,
   '2': -111154.78071310197,
   '5': -96249.65747978249,
   '6': -101370.67964817032},
  'policy_reward_mean': {'1': -102445.36977694454,
   '2': -111154.78071310197,
   '5': -96249.65747978249,
   '6': -101370.67964817032},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-411220.48761799914],
   'episode_lengths': [720],
   'policy_1_reward': [-102445.36977694454],
   'policy_2_reward': [-111154.78071310197],
   'policy_5_reward': [-96249.65747978249],
   'policy_6_reward': [-101370.67964817032]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 8.444797976172417,


In [11]:
ray.shutdown()

## Train RL Agent

In [6]:
ray.init()

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(train_env_creator(config)))

2023-06-04 12:00:53,481	INFO worker.py:1625 -- Started a local Ray instance.


In [7]:
algo = config.build()

2023-06-04 12:01:00,766	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-04 12:01:00,797	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=25104) delta_wait_time_reward 0.7
(RolloutWorker pid=25104) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                      


(RolloutWorker pid=10800) 2023-06-04 12:01:07,271	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-04 12:01:10,970	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=6728) delta_wait_time_reward 0.7 [repeated 10x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=32544) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 9x across cluster]


(RolloutWorker pid=6728) 2023-06-04 12:01:14,779	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-04 12:01:14,867	INFO trainable.py:172 -- Trainable.setup took 14.075 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [8]:
from datetime import datetime

TRAIN_EPS = 1400  # 720 * 1400 == 1_008_000 total timesteps
CHECKPOINT_FREQ = 100
assert TRAIN_EPS % CHECKPOINT_FREQ == 0

tic = datetime.now()

for i in range(TRAIN_EPS):
    results = algo.train()

    if (i+1) % CHECKPOINT_FREQ == 0:
        algo.save(os.path.join("ray_checkpoints","grid2x2",f"test_{TEST_NUM}"))

toc = datetime.now()

2023-06-04 12:01:28,873	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Step #3600.00 (1ms ~= 1000.00*RT, ~120000.00UPS, TraCI: 103ms, vehicles TOT 2106 ACT 120 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~120000.00UPS, TraCI: 103ms, vehicles TOT 2106 ACT 120 BT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~140000.00UPS, TraCI: 108ms, vehicles TOT 2088 ACT 140 BT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~141000.00UPS, TraCI: 104ms, vehicles TOT 2098 ACT 141 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~165000.00UPS, TraCI: 106ms, vehicles TOT 2102 ACT 165 B24ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~166000.00UPS, TraCI: 102ms, vehicles TOT 2081 ACT 166 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~175000.00UPS, TraCI: 91ms, vehicles TOT 2022 ACT 175 BUOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~201000.00UPS, TraCI: 100ms, vehicles TOT 1578 ACT 201 BOT

(RolloutWorker pid=22940) 2023-06-04 12:37:16,736	ERROR actor_manager.py:179 -- Worker exception, recreating: unpack requires a buffer of 3 bytes
(RolloutWorker pid=22940) Traceback (most recent call last):
(RolloutWorker pid=22940)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
(RolloutWorker pid=22940)     return func(self, *args, **kwargs)
(RolloutWorker pid=22940)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\execution\rollout_ops.py", line 86, in <lambda>
(RolloutWorker pid=22940)     lambda w: w.sample(), local_worker=False, healthy_only=True
(RolloutWorker pid=22940)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
(RolloutWorker pid=22940)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=22940)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 

(RolloutWorker pid=21904) delta_wait_time_reward 0.7
(RolloutWorker pid=21904) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                     
Step #3600.00 (1ms ~= 1000.00*RT, ~52000.00UPS, TraCI: 92ms, vehicles TOT 2106 ACT 52 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~55000.00UPS, TraCI: 83ms, vehicles TOT 2106 ACT 55 BUF 18ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~57000.00UPS, TraCI: 87ms, vehicles TOT 2106 ACT 57 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~55000.00UPS, TraCI: 81ms, vehicles TOT 2106 ACT 55 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 79ms, vehicles TOT 2106 ACT 53 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 75ms, vehicles TOT 2106 ACT 55 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 67ms, vehicles

(RolloutWorker pid=25104) Error: tcpip::Socket::accept() Unable to create listening socket: Address already in use
(RolloutWorker pid=25104) Quitting (on error).


...

(RolloutWorker pid=9600) Could not connect to TraCI server at localhost:50957 [WinError 10061] No connection could be made because the target machine actively refused it


2023-06-04 12:47:33,330	WARNING worker.py:1986 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff21d9d490f881450fadf79db101000000 Worker ID: 44adbcb479fc395dbb6608adec89131dc2ca16946e84383e66f0bf5e Node ID: b1121a1d1a0fa53114aa61251cb7088dc1e2774fbbaf36fd9e6c1036 Worker IP address: 127.0.0.1 Worker port: 50150 Worker PID: 9600 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code 1.
(RolloutWorker pid=9600) 2023-06-04 12:47:33,325	ERROR worker.py:844 -- Worker exits with an exit code 1.
(RolloutWorker pid=9600) Traceback (most recent call last):
(RolloutWorker pid=9600)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
(RolloutWorker pid=9600)     return func(self, *args, **kwargs)
(RolloutWorker pid=9600)   File "c:\Users\admin\anaconda3\env

(RolloutWorker pid=32220) delta_wait_time_reward 0.7
(RolloutWorker pid=32220) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 76ms, vehicles TOT 2106 ACT 53 BUF 0)               17ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 80ms, vehicles TOT 2106 ACT 51 BUF 0)               1ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~47000.00UPS, TraCI: 70ms, vehicles TOT 2106 ACT 47 BUF 17ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~48000.00UPS, TraCI: 75ms, vehicles TOT 2106 ACT 48 BUF T 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~54000.00UPS, TraCI: 74ms, vehicles TOT 2106 ACT 54 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 80ms, vehicles TOT 2106 ACT 55 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 72ms, vehicles 

(RolloutWorker pid=15496) 2023-06-04 16:44:31,810	ERROR actor_manager.py:179 -- Worker exception, recreating: unpack requires a buffer of 3 bytes
(RolloutWorker pid=15496) Traceback (most recent call last):
(RolloutWorker pid=15496)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
(RolloutWorker pid=15496)     return func(self, *args, **kwargs)
(RolloutWorker pid=15496)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\execution\rollout_ops.py", line 86, in <lambda>
(RolloutWorker pid=15496)     lambda w: w.sample(), local_worker=False, healthy_only=True
(RolloutWorker pid=15496)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
(RolloutWorker pid=15496)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=15496)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 

(RolloutWorker pid=28580) delta_wait_time_reward 0.7
(RolloutWorker pid=28580) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 74ms, vehicles TOT 2106 ACT 49 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 74ms, vehicles TOT 2106 ACT 49 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 74ms, vehicles TOT 2106 ACT 49 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 75ms, vehicles TOT 2106 ACT 47 BUF 0)               26ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 88ms, vehicles TOT 2106 ACT 52 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 84ms, vehicles TOT 2106 ACT 46 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 87ms, vehicles 

In [9]:
str(toc - tic)

'5:56:30.867903'

In [10]:
ray.shutdown()

(RolloutWorker pid=22304) 2023-06-04 16:45:31,825	ERROR worker.py:844 -- Worker exits with an exit code 1.
(RolloutWorker pid=22304)     sys.exit(1) [repeated 3x across cluster]
(RolloutWorker pid=22304) During handling of the above exception, another exception occurred:
(RolloutWorker pid=22304)   File "python\ray\_raylet.pyx", line 1197, in ray._raylet.task_execution_handler
(RolloutWorker pid=22304)   File "python\ray\_raylet.pyx", line 1100, in ray._raylet.execute_task_with_cancellation_handler
(RolloutWorker pid=22304)   File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task [repeated 4x across cluster]
(RolloutWorker pid=22304)   File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=22304)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
(RolloutWorker pid=22304)     return method(__ray_actor, *args, **kwargs)
(RolloutWorker pid=22304

## Play Trained Agent

In [16]:
register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

In [17]:
ray.init()

2023-06-04 18:24:19,245	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.8.16
Ray version:,2.4.0


In [18]:
from ray.rllib.algorithms.ppo import PPO

checkpoint_path = os.path.join("ray_checkpoints",ENV_NAME,f"test_{TEST_NUM}",f"checkpoint_{TRAIN_EPS:06}")
checkpoint_path = os.path.abspath(checkpoint_path)
print(checkpoint_path)

ppo_agent = PPO.from_checkpoint(checkpoint_path)

2023-06-04 18:24:22,854	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


c:\Users\admin\masters-proj\ray_checkpoints\grid2x2\test_6\checkpoint_001400
(RolloutWorker pid=29016) delta_wait_time_reward 0.7
(RolloutWorker pid=29016) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 0 ACT 0 BUF 0)                      


(RolloutWorker pid=30232) 2023-06-04 18:24:29,424	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-04 18:24:29,689	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-04 18:24:33,799	INFO trainable.py:172 -- Trainable.setup took 10.941 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [19]:
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_trained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure",])

In [20]:
ppo_agent.evaluate()

Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 162ms, vehicles TOT 2106 ACT 46 BUF 0)              OT 3 ACT 3 BUF 0)                      
(RolloutWorker pid=11936) delta_wait_time_reward 0.7 [repeated 10x across cluster]
(RolloutWorker pid=11936) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 10x across cluster]


{'evaluation': {'episode_reward_max': -81850.74284503894,
  'episode_reward_min': -81850.74284503894,
  'episode_reward_mean': -81850.74284503894,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'1': -20433.59504688408,
   '2': -20982.19554992575,
   '5': -19240.044754114908,
   '6': -21194.907494114097},
  'policy_reward_max': {'1': -20433.59504688408,
   '2': -20982.19554992575,
   '5': -19240.044754114908,
   '6': -21194.907494114097},
  'policy_reward_mean': {'1': -20433.59504688408,
   '2': -20982.19554992575,
   '5': -19240.044754114908,
   '6': -21194.907494114097},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-81850.74284503894],
   'episode_lengths': [720],
   'policy_1_reward': [-20433.59504688408],
   'policy_2_reward': [-20982.19554992575],
   'policy_5_reward': [-19240.044754114908],
   'policy_6_reward': [-21194.907494114097]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 8.388285828694887,
   'mean

In [21]:
ray.shutdown()

(RolloutWorker pid=11936) 2023-06-04 18:24:33,680	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
